In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import os
from torchvision import transforms,datasets
from torch.utils.data import Dataset, DataLoader
# import torch.utils.data
import torch.optim as optim
import torch
import cv2
from PIL import Image
from torch.autograd import Variable

In [1]:
import numpy as np

In [9]:
a = np.array([1,10])

In [11]:
b = np.array([1,5])

In [12]:
a == b

array([ True, False])

In [14]:
a^b

array([ 0, 15], dtype=int32)

In [10]:
for i in a:
    print(i)

1
10


In [3]:
a

array(10)

In [ ]:
![](picture/1.png)

In [ ]:
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=True):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1),
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta


    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x


In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()

        # conv
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)

        # LRN
        self.LRN = LRN(local_size=5, alpha=0.0001, beta=0.75)

        # FC
        self.fc1 = nn.Linear(256*6*6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2)

        # Dropout
        self.Dropout = nn.Dropout()



    def forward(self, x):
        # conv1 -> relu -> maxpool1
        # conv1: [n, 3, 227, 227] --> [n, 96, 55, 55]
        # maxpool1: [n, 96, 55, 55] --> [n, 96, 27, 27]
        x = F.relu(self.conv1(x))
        x = self.LRN(x)
        x = F.max_pool2d(x, (3, 3), 2)

        # conv2 -> relu -> maxpool2
        # conv2: [n, 96, 27, 27] --> [n, 256, 27, 27]
        # maxpool2: [n, 256, 27, 27] --> [n, 256, 13, 13]
        x = F.relu(self.conv2(x))
        x = self.LRN(x)
        x = F.max_pool2d(x, (3, 3), 2)

        # conv3 -> relu -> conv4 -> relu
        # oonv3: [n, 256, 13, 13] --> [n, 384, 13, 13]
        # conv4: [n, 384, 13, 13] --> [n, 384, 13, 13]
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        # conv5 -> relu -> maxpool3
        # conv5: [n. 384, 13, 13] --> [n, 256, 13, 13]
        # maxpool3: [n, 256, 13, 13] --> [n, 256, 6, 6]
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, (3, 3), 2)

        # need view first for conv --> FC
        x = x.view(x.size()[0], -1)

        # fc1 -> fc2 -> fc3 -> softmax
        # fc1: 256*6*6 --> 4096
        # fc2: 4096 --> 4096
        # fc3: 1000 --> 2
        x = F.relu(self.fc1(x))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)
        x = F.softmax(x)
        return x

In [ ]:
#数据预处理
pre_transforms = transforms.Compose([#一组操作组合起来
    transforms.Resize((227,227)),
    transforms.ToTensor(),#转换类型
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self,args,mode = 'train',transform = None):
        self.args = args
        self.transform = transform
        self.mode = mode
        self.names = self.__dataset_info()
    def __getitem__(self,index):#读取一个数据
        x = cv2.imread(self.args+'/'+self.names[index],1)
        #x 为numpy ndarry
        x = Image.fromarray(x)#PIL
        # array -> image
        x_label = 0 if 'cat' in self.names[index] else 1 #贴标签
        if self.transform is not None:
            x = self.transform(x)
        return x,x_label
    
    def __len__(self):
        return len(self.names)
    def __dataset_info(self):#读取数据
        img_path = self.args
        imgs = [f for f in os.listdir(img_path) if
                os.path.isfile(os.path.join(img_path,f)) and f.endswith('.jpg')]
    #listdir 返回当前文件夹下的文件名
    #os.path.join 将目录和文件名合并成一个路径
    #os.path.isfile 判断是否为文件
    #f为字符串
        names = []
        for name in imgs:
            index = int(name.split('.')[1])
            #train dataset
            if self.mode == 'train':
                if index >= 500:
                    names.append(name)
            #test dataset:1000 cat1 dog1
            elif self.mode == 'test':
                if index < 500:
                    names.append(name)
        return names

In [ ]:
# 实例化 从训练数据中，划分训练集和验证集
train_dataset = CatDogDataset('/kaggle/input/cat-dog/train/train/','train',pre_transforms)
test_dataset = CatDogDataset('/kaggle/input/cat-dog/train/train/','test',pre_transforms)

#print the length of train_dataset
print('train:{} img'.format(len(train_dataset)))
print('test:{} img'.format(len(test_dataset)))

In [ ]:
batch_size = 100
train_loader = DataLoader(train_dataset,batch_size,shuffle = True)
test_loader = DataLoader(test_dataset,1,shuffle = False)

In [ ]:
alexnet = AlexNet()

In [ ]:
#损失函数与优化方法
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(),#momentum 动量
                lr = 0.01,momentum = 0.9,weight_decay = 5e-4)

In [ ]:
#选择加载到哪个设备上
#选择运行在哪个设备上
# if :
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     print('using CPU!')
# else:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU')
else:
    device = torch.device('cpu')
    print('using CPU')


In [ ]:
epochs = 10
# base_epoch = 0
alexnet = alexnet.to(device)
for epoch in range(epochs):
    alexnet.train()
    # epoch += base_epoch
    epoch_loss = 0
    for idx,(imgs,labels) in enumerate(train_loader):
        imgs,labels = imgs.to(device),labels.to(device)
        pre_labels = alexnet(imgs)#预测标签
        optimizer.zero_grad()
        loss = criterion(pre_labels,labels.long())#计算损失
        loss.backward()
        optimizer.step()#更新参数

        epoch_loss += loss.item()

        print('[{}/{}][{}/{}] loss:{:.4f}'
                  .format(epoch+1, epochs, idx+1, int(len(train_dataset) / batch_size), loss.item()))

In [ ]:
def get_accuracy(model,data_loader):
    correct_pred = 0
    n = 0
    with torch.no_grad():#反向传播不求偏导数
        model.eval()
        for X,y_true in data_loader:
            X = X.to(device)
            y_pred = model(X)
            predicted_labels = torch.max(y_pred,1)[1].item()#data dim
#             print(predicted_labels)
            
            n+=y_true.size(0)
            y_true = y_true.item()
            correct_pred += (predicted_labels == y_true)
#             print('{},{}'.format(n,correct_pred))
#             print(type(predicted_labels))
#             print(type(y_true))
#             break
    return float(correct_pred)/n

In [ ]:
# test_loader2 = test_loader.to('cuda')
acc = get_accuracy(alexnet,test_loader)
print('模型准确度{:.2f}'.format(acc))
# inputs = inputs.to(device)

In [ ]:
torch.save({'model': alexnet.state_dict()}, 'model_name.pth')

In [ ]:
torch.save(alexnet.state_dict(),'model.pt')